# Computer Vision (3rd Notebook)

# 41.Vehicle detection in a Video frame


In [1]:
import cv2
import numpy as np

In [42]:
import cv2

# Load video
cap = cv2.VideoCapture(r'E:\Computer-Vision\Images\Vehical_Run.mp4')

# Load the car classifier once
car_cascade = cv2.CascadeClassifier(r'E:\Computer-Vision\haarcascade_vehicle.xml')

while True:
    ret, frame = cap.read()
    
    if ret:
        # Convert frame to grayscale
        frame = cv2.resize(frame, (700,500))
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Detect cars
        cars = car_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=3)
        
        # Draw rectangles around detected cars
        for (x, y, w, h) in cars:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 1)
        
        # Display the frame
        cv2.imshow('Video Frame', frame)
        
        # Break the loop if 'q' is pressed
        if cv2.waitKey(20) & 0xFF == ord('q'):
            break
    else:
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


# 42. Smile and Eye detection

In [20]:
image =cv2.imread(r'E:\Computer-Vision\Images\Uzair.jpg')
image =cv2.resize(image,(500,500))


gry =cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
sm =cv2.CascadeClassifier(r'E:\Computer-Vision\haarcascade_smile.xml')
fc =cv2.CascadeClassifier(r'E:\Computer-Vision\haarcascade_frontalface_default.xml')

f = fc.detectMultiScale(gry,1.3,3)

for x,y,w,h in f:
    roi_gry =gry[y:y+h,x:x+w]
    roi_img =image[y:y+h,x:x+w]
    
    s =sm.detectMultiScale(roi_img,1.5,5)
    
    for xs,ys,ws,hs in s:
        cv2.rectangle(roi_img,(xs,ys),(xs+ws,ys+hs),(0,0,255),2)
        

cv2.imshow('My Image',image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 43.Pedestrian detection from a streaming video

In [16]:
import cv2

# Load video
cap = cv2.VideoCapture(r'E:\Computer-Vision\Images\Person_detection_video.mp4')

# Load Haar cascade once
hm = cv2.CascadeClassifier(r'E:\Computer-Vision\haarcascade_fullbody.xml')

# Check if cascade loaded
if hm.empty():
    print("Error loading Haar cascade")
    exit()

while True:
    r, frame = cap.read()
    
    if not r:
        break  # End of video

    # Resize frame
    frame = cv2.resize(frame, (700, 500))

    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect people
    bodies = hm.detectMultiScale(gray, 1.1, 5)

    # Draw rectangles and label each detected person
    for x, y, w, h in bodies:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
        cv2.putText(frame, 'Person', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX,
                    0.3, (255, 0, 0), 1, cv2.LINE_AA)

    # show total number of people detected
    cv2.putText(frame, f'Total Persons: {len(bodies)}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
                0.8, (0, 0, 255), 2, cv2.LINE_AA)

    # Show frame
    cv2.imshow('Person Detection', frame)

    # Break on 'q' key
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


# 44.MediaPipe (Face Detection)
MediaPipe is a cross-platform, open-source framework by Google for building real-time perception pipelines. It’s widely used in computer vision tasks like:

Face detection | Face Mesh | Iris | Hands Gesture recognition| Pose | Holistic full Body | Hair Segmentation | Object Detection | KNIFT written data | Box Tracking | Instant Motion Tracking 

In [2]:
import mediapipe as mp
print("✅ mediapipe imported successfully!")


✅ mediapipe imported successfully!


In [3]:
mp_face_det = mp.solutions.face_detection
mp_draw =mp.solutions.drawing_utils

In [31]:

# Confidence 1.0 (100%) sirf perfect detections ko allow karega, is liye koi face detect na ho sakta hai.
# Isko 0.5 rakho taake realistic result mile.
face_dect = mp_face_det.FaceDetection(min_detection_confidence=0.5, model_selection=0)

# Video file load
cap = cv2.VideoCapture(r'E:\Computer-Vision\Images\Face_detection.mp4')

# Check if video opened correctly
if not cap.isOpened():
    print("❌ Error: Video file not opened. Check path!")
    exit()

while True:
    r, frame = cap.read()
    if not r:   # agar frame read na ho
        print("End of video or cannot read frame")
        break

    # BGR → RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = face_dect.process(rgb_frame)

    # Back RGB → BGR
    frame = cv2.cvtColor(rgb_frame, cv2.COLOR_RGB2BGR)
    frame = cv2.flip(frame, 1)

    # Agar detections hain to draw karo
    if result.detections:
        for id, detection in enumerate(result.detections):
            mp_draw.draw_detection(frame, detection)

    # Show frame
    cv2.imshow('Video Frame', frame)

    # Quit key
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


# 45.MediaPipe  Hand Tracking 

In [65]:
import mediapipe as mp
import cv2

mp_drawing = mp.solutions.drawing_utils
mp_drawing.styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

cap = cv2.VideoCapture(r'E:\Computer-Vision\Images\Hands_Tracking_Video.mp4')
p = mp_hands.Hands(model_complexity=0, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Finger tip landmarks (Thumb, Index, Middle, Ring, Pinky)
finger_tips = [4, 8, 12, 16, 20]

while cap.isOpened():
    r, frame = cap.read()
    if r == True:
        # frame = cv2.flip(frame, 1)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        result = p.process(frame)

        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

        if result.multi_hand_landmarks:
            for hand_landmarks in result.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame,
                                          hand_landmarks,
                                          mp_hands.HAND_CONNECTIONS)

                fingers_up = 0
                landmarks = hand_landmarks.landmark

                h, w, _ = frame.shape

                # Thumb check (x-axis compare because thumb moves sideways)
                if landmarks[finger_tips[0]].x > landmarks[finger_tips[0] - 1].x:
                    fingers_up += 1

                # Other 4 fingers check (y-axis compare because fingers move up-down)
                for tip in finger_tips[1:]:
                    if landmarks[tip].y < landmarks[tip - 2].y:
                        fingers_up += 1

                # Show text on top
                cv2.putText(frame, f"Fingers: {fingers_up}", (30, 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 0), 3)

        cv2.imshow('Hand Tracking', frame)
        if cv2.waitKey(25) & 0xff == ord('q'):
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()
